# Iport Dependencies

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from datetime import datetime
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor
from statsmodels.tsa.arima.model import ARIMA
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import uniform, randint
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import gc
import lightgbm as lgb
import optuna
%matplotlib inline

Libraries imported


# Load Dataset

In [5]:
cc_train = pd.read_csv('raw_data/train_data.csv')
cc_test = pd.read_csv('raw_data/test_data.csv')


# Reduce Memory Usage

In [11]:
def reduce_mem_usage(dataframe, verbose=True):
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_memory = dataframe.memory_usage().sum() / 1024**2
  for col in dataframe.columns:
    col_type = dataframe[col].dtypes
    if col_type in numerics:
      c_min = dataframe[col].min()
      c_max = dataframe[col].max()
      if str(col_type)[:3] == 'int':
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
          dataframe[col] = dataframe[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
          dataframe[col] = dataframe[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
          dataframe[col] = dataframe[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
          dataframe[col] = dataframe[col].astype(np.int64)
      else:
        if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
          dataframe[col] = dataframe[col].astype(np.float32)
        else:
          dataframe[col] = dataframe[col].astype(np.float64)
  end_memory = dataframe.memory_usage().sum() / 1024**2
  print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_memory, 100 * (start_memory - end_memory) / start_memory)) if verbose else print('Reduced to {:5.2f}'.format(end_memory))
  return dataframe

In [12]:
cc_train = reduce_mem_usage(cc_train)

Mem. usage decreased to 352.24 Mb (50.1% reduction)


# Outliers

In [13]:
train_df = cc_train.copy()
test_df = cc_test.copy()

In [14]:
from scipy.stats import iqr

def remove_outliers_tukey(data, alpha=1.5):
    '''
    Remove outliers using Tukey's method with the interquartile range (IQR).
    
    Parameters:
    data (numpy array or pandas dataframe): The data to remove outliers from.
    alpha (float): The sensitivity parameter, which determines the range to consider outliers.
                   A value of 1.5 is the default, which is a commonly used value.
    
    Returns:
    numpy array or pandas dataframe: The data with outliers removed.
    '''
    # Select only the numerical columns
    num_cols = data.select_dtypes(include=[np.number]).columns
    data_num = data[num_cols]
    
    # Compute the first and third quartiles
    q1, q3 = np.percentile(data_num, [25, 75])
    
    # Compute the interquartile range (IQR)
    iqr_val = iqr(data_num)
    
    # Compute the range outside of which data points are considered outliers
    outlier_range = (q1 - alpha * iqr_val, q3 + alpha * iqr_val)
    
    # Identify the outliers and remove them
    outliers = (data_num < outlier_range[0]) | (data_num > outlier_range[1])
    data_num_no_outliers = data_num[~outliers]
    
    # Merge the numerical columns back into the original data frame
    data_no_outliers = pd.concat([data_num_no_outliers, data.select_dtypes(exclude=[np.number])], axis=1)
    return data_no_outliers

data_with_removed_outliers = remove_outliers_tukey(train_df)
data_with_removed_outliers.shape

(375734, 246)

# Preprocessing

In [15]:
target="contest-tmp2m-14d__tmp2m"

In [16]:
def location_feature(train, test):
    scale = 14
    train.loc[:,'lat']=round(train.lat,scale)
    train.loc[:,'lon']=round(train.lon,scale)
    test.loc[:,'lat']=round(test.lat,scale)
    test.loc[:,'lon']=round(test.lon,scale)
    
    train_and_test = pd.concat([train, test], axis=0)
    train_and_test['loc_group'] = train_and_test.groupby(['lat', 'lon']).ngroup()
    print(f'{train_and_test.loc_group.nunique()} unique locations')
    
    train = train_and_test.iloc[:len(train)]
    test = train_and_test.iloc[len(train):].drop(target, axis=1)
    
    return train, test

def cat_encode(train, test):
    # encoding the categorical feature in the train and test data set
    # using OneHotEncoder
    ohe = OneHotEncoder()
    train_encoded = ohe.fit_transform(train[['climateregions__climateregion']])
    test_encoded = ohe.transform(test[['climateregions__climateregion']])
    
    train = train.drop(['climateregions__climateregion'], axis=1)
    test = test.drop(['climateregions__climateregion'], axis=1)
    
    train_encoded = pd.DataFrame(train_encoded.toarray(), columns=ohe.get_feature_names_out(['climateregions__climateregion']))
    test_encoded = pd.DataFrame(test_encoded.toarray(), columns=ohe.get_feature_names_out(['climateregions__climateregion']))
    
    train = pd.concat([train, train_encoded], axis=1)
    test = pd.concat([test, test_encoded], axis=1)
    
    return train, test

    
def fill_na_rows(dataset):
    # Find the columns with missing values
    columns_with_missing_values = dataset.columns[dataset.isnull().any()].tolist()
    
    # Impute the missing values with the mean value of that column
    for col in columns_with_missing_values:
        dataset[col].fillna(dataset[col].mean(), inplace=True)
        
    return dataset

def create_new_feat(dataset):
    dataset['year']=pd.DatetimeIndex(dataset['startdate']).year 
    dataset['month']=pd.DatetimeIndex(dataset['startdate']).month 
    dataset['day']=pd.DatetimeIndex(dataset['startdate']).day
    return dataset

def feature_engineering(origin_train, origin_test):
    train, test = origin_train, origin_test
    train = fill_na_rows(train)
    train = create_new_feat(train)
    test = create_new_feat(test)
    train, test = cat_encode(train, test)
    irrelevant_cols = ['index', 'startdate','contest-tmp2m-14d__tmp2m', 'climateregions__climateregion']
    features = [col for col in train.columns if col not in irrelevant_cols]
    #features = [col for col in train.columns]
    X = train[features]
    X_test = test[features]
    y = train['contest-tmp2m-14d__tmp2m']
    # Initialize the scaler
    #scaler = MinMaxScaler()

    # Fit the scaler to the train data
    #scaler.fit(X)

    # Transform the train data
    #X_train_scaled = scaler.transform(X)

    # Transform the test data
    #X_test_scaled = scaler.transform(X_test)
    
    return X, y, X_test



In [17]:
cc_test_copy = cc_test.copy()
X, y, X_test = feature_engineering(cc_train.copy(), cc_test.copy())

In [20]:
def identify_correlated(df, threshold):
    corr_matrix = df.corr().abs()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    reduced_corr_matrix = corr_matrix.mask(mask)
    features_to_drop = [c for c in reduced_corr_matrix.columns if any(reduced_corr_matrix[c] > threshold)]
    return features_to_drop

In [21]:
features_to_drop = identify_correlated(cc_train, .80)

In [25]:
remove_feature = ['index', 'contest-tmp2m-14d__tmp2m']
features_to_drop_v1 = [ele for ele in features_to_drop if ele not in remove_feature]


In [26]:
cc_train_reduced = pd.DataFrame(X.drop(features_to_drop_v1, axis=1))
cc_test_reduced = pd.DataFrame(X_test.drop(features_to_drop_v1, axis=1))

In [27]:
def ensemble_preds(xgboost_preds, light_gbm_preds):
    combined_preds = np.mean([xgboost_preds, lightgbm_preds], axis=0)
    return combined_preds

# Model

In [28]:
X_train, X_test_tts, y_train, y_test = train_test_split(cc_train_reduced, y, test_size=0.33, random_state=42)

## Random Forest

In [29]:
params = {
    'n_estimators': 5000,
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'sqrt',
    'bootstrap': True,
    'oob_score': True
}
regr_rfr = RandomForestRegressor(**params)
regr_rfr.fit(X_train, y_train)
# make predictions on the test data
y_pred_rfr = regr_rfr.predict(X_test_tts)

# calculate the RMSE
rmse = mean_squared_error(y_test, y_pred_rfr,squared=False)
print("RMSE:", rmse)

KeyboardInterrupt: 

In [ ]:
cc_test_pred = regr_rfr.predict(cc_test_reduced)

In [ ]:
cc_test_copy[target] = cc_test_pred
cc_test_copy[[target,"index"]].to_csv("rfrpredictions.csv",index = False)

## XGBoost

In [ ]:
def objective(trial):
    params = {
            'base_score': 0.5, 
            'booster': 'gbtree',
            'tree_method': 'gpu_hist',
            'n_estimators': 6000,
            'objective': 'reg:squarederror',
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'subsample': trial.suggest_uniform('subsample', 0.1, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1.0),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
            'verbosity': -1,
            'gpu_id': 0
        }
        reg_xgb = xgb.XGBRegressor(**params)

        reg_xgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test_tts, y_test)])

        y_pred_xgb = reg_xgb.predict(X_test_tts)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

        return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=15)

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print(f'  Value: {trial.value:.5f}')
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

best_params = trial.params
reg_xgb = xgb.XGBRegressor(**best_params)
reg_xgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test_tts, y_test)])
y_pred_xgb = reg_xgb.predict(X_test_tts)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print("RMSE:", rmse)

cc_test_pred = reg_xgb.predict(cc_test_reduced)
cc_test_copy[target] = cc_test_pred
cc_test_copy[[target,"index"]].to_csv("xgbpredictions.csv",index = False)
print("Finished training and fitting, created xgbpredictions.csv")


# Robust Linear

In [ ]:
import statsmodels.api as sm

# Fit a robust linear regression model
rlm_model = sm.RLM(y_train, X_train, M=sm.robust.norms.Hampel())
rlm_results = rlm_model.fit(scale_est=sm.robust.scale.HuberScale())

# Print the summary of the model
print(rlm_results.summary())

# Get the predicted values on the test set
y_pred = rlm_results.predict(X_test_tts)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)